In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, LeakyReLU, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings('ignore')

**Loading Test and Train Dataset**

In [ ]:
train_data = pd.read_csv("../input/digit-recognizer/train.csv")
test_data = pd.read_csv("../input/digit-recognizer/test.csv")

**Ploting Image**

In [ ]:
def show_image(train_image, label, index):
    image_shaped = train_image.values.reshape(28,28)
    plt.subplot(3, 6, index+1)
    plt.imshow(image_shaped, cmap=plt.cm.gray)
    plt.title(label)


plt.figure(figsize=(18, 8))
sample_image = train_data.sample(18).reset_index(drop=True)
for index, row in sample_image.iterrows():
    label = row['label']
    image_pixels = row.drop('label')
    show_image(image_pixels, label, index)
plt.tight_layout()

**Seperating Lable and Pixels**

In [ ]:
x = train_data.drop(columns=['label']).values.reshape(train_data.shape[0],28,28,1)
y = to_categorical(train_data['label']) 

**Normalazing (0 to 255) to (0 to 1)**

In [ ]:
x = x/255

**Splitting Data for Validation**

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((x, y))
train_ds = train_ds.shuffle(2500)

val_ds = tf.data.Dataset.from_tensor_slices((x, y))
val_ds = val_ds.shuffle(2500)

In [ ]:
train_ds = train_ds.skip(8000).batch(32)
val_ds = val_ds.take(8000).batch(32)

In [ ]:
train_ds

***Model Build***

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(28, 28,1)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.002), loss='categorical_crossentropy', metrics=['accuracy'])

**Callback to save the best model**

In [ ]:
call_back = ModelCheckpoint('model.h5', verbose=1, monitor='val_accuracy', save_best_only=True)

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=15, callbacks=[call_back])

**Loading the best model**

In [ ]:
model = tf.keras.models.load_model('./model.h5')

**Preparing test Data**

In [ ]:
test_data = test_data.values.reshape(test_data.shape[0],28,28,1)
test_data = test_data / 255

**Prediction**

In [ ]:
pred = model.predict(test_data, verbose=1)

In [ ]:
predicted = []
for i in range(len(pred.round())):
    predicted.append(np.argmax(pred.round()[i]))

**Submission**

In [ ]:
submission_file = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
submission_file['Label'] = predicted
submission_file.to_csv('MySubmission.csv', index=False)